
## bacteria archea traits DH template 

In [ ]:
from linkml_runtime.utils.schemaview import SchemaView
import pandas as pd

In [ ]:
bat_yaml_file = (
    "../../icbo2021/linkml-model-enrichment/condensed_traits_NCBI_tidied_curated.yaml"
)

rows_list = []
headers_list = []

ultilized_enums = {}

blank_row = {}

default_section = "Traits"

headers_list.append({"label": default_section})

In [ ]:
bat_yaml_view = SchemaView(bat_yaml_file)

bat_classes = bat_yaml_view.all_class()

bat_classes_names = list(bat_classes.keys())

bat_enums = bat_yaml_view.all_enum()
bat_enum_names = list(bat_enums.keys())

# assuming single class from linkml model enrichment
# DH only supports one class anyway
class_slots = bat_yaml_view.class_slots(bat_classes_names[0])

bat_slots = bat_yaml_view.all_slot()

In [ ]:
rows_list = []
utilized_enums = {}

for i in class_slots:
    row_dict = blank_row.copy()
    row_dict["Ontology ID"] = ""
    row_dict["parent class"] = default_section
    row_dict["label"] = i
    row_dict["datatype"] = "xs:token"
    row_dict["source"] = ""
    row_dict["data status"] = ""
    row_dict["requirement"] = ""
    row_dict["min value"] = ""
    row_dict["max value"] = ""
    row_dict["capitalize"] = ""
    row_dict["description"] = ""
    row_dict["guidance"] = ""

    slot_obj = bat_slots[i]
    # assuming single example from linkml model enrichment
    current_example = slot_obj.examples[0].value
    row_dict["examples"] = current_example
    current_range = slot_obj.range
    if current_range == "integer":
        # constraint becomes more restrictive
        row_dict["datatype"] = "xs:nonNegativeInteger"
    if current_range == "float":
        row_dict["datatype"] = "xs:decimal"
    if current_range == "datetime":
        row_dict["datatype"] = "xs:date"
    takes_enum = slot_obj.range in bat_enum_names
    if takes_enum:
        permissibles_count = len(bat_enums["cell_shape_enum"].permissible_values)
        # don't bother making select for a single value
        if permissibles_count >= 2:
            # assuming no multi valueds
            row_dict["datatype"] = "select"
            current_pvs = bat_enums[slot_obj.range].permissible_values
            enum_permissibles = []
            for k, v in current_pvs.items():
                current_meaning = v.meaning
                current_description = v.description
                assembled_meaning = current_description
                if current_meaning is not None:
                    assembled_meaning = (
                        current_description + " (" + current_meaning + ")"
                    )
                enum_permissibles.append(assembled_meaning)
                enum_permissibles.sort()
        utilized_enums[i] = enum_permissibles
    rows_list.append(row_dict)

rows_list = headers_list + rows_list

rows_frame = pd.DataFrame(rows_list)

# utilized_enums

In [ ]:
reshape_list = []
for enum_name, permissibles in utilized_enums.items():
    #     print(enum_name)
    for permissible in permissibles:
        reshape_list.append({"parent class": enum_name, "label": permissible})

reshape_frame = pd.DataFrame(reshape_list)
# reshape_frame

In [ ]:
package_template = pd.concat([rows_frame, reshape_frame])
package_template = package_template.fillna("")

package_template

In [ ]:
# package_template.to_csv("data.tsv", sep="\t", index=False)